# IPC Alerts of Acute Food Insecurity
Capture current IPC alerts as displayed on this ipcinfo.org [page](http://www.ipcinfo.org/ipcinfo-website/resources/alerts-archive/en/).

In [1]:
# General imports
# helpers
from pathlib import Path
import urllib.request
from datetime import datetime
from tqdm import tqdm
import copy

# data processing
import pandas as pd
import requests
from urllib.request import Request, urlopen

from bs4 import BeautifulSoup

# conf
import sys
from path_manipulation import get_to_root

sys.path[0] = get_to_root(3,sys.path[0])
from config.config import config
sources = config.sources

## Finding all alerts, then finding all related documents

In [2]:
def get_all_alerts_hyperlinks(url):
    # Find all hyperlinks for alerts on the page,
    headers = {"User-Agent": "Mozilla/5.0"}
    req = Request(url=url, headers=headers)
    with urllib.request.urlopen(req) as page:

        # Parse .xml file
        xml = BeautifulSoup(page.read(), "lxml")

        # Get all dataset info
        links = []
        for link in xml.findAll("a"):
            title = link.get("title")
            url = link.get("href")
            if (
                (title is not None)
                and (title != "image thumbnail")
                and (url.startswith("/ipcinfo-website/alerts-archive/issue-"))
            ):
                url = "http://www.ipcinfo.org" + url
                links.append({"title": title, "url": url})
    # TODO: add the alerts paragraph to the metadata
    return links

In [3]:
def get_pdf_from_url(url, output_path=None):
    """Download a pdf from a given url.
    :param url: web url from where to download the pdf
    :param output_path: a pathlib.Path object to which to write the downloaded pdf
    """
    # create the folder if not existing
    output_path.parent.mkdir(parents=True, exist_ok=True)
    # download
    r = requests.get(url)
    with open(output_path, "wb") as outfile:
        outfile.write(r.content)


def get_all_documents_from_alerts_references(links):
    documents = []
    for link in tqdm(links):
        headers = {"User-Agent": "Mozilla/5.0"}
        req = Request(url=link["url"], headers=headers)

        with urllib.request.urlopen(req) as page:
            xml = BeautifulSoup(page.read(), "lxml")
            alert_page_title = xml.find("div", class_="csc-header csc-header-n2").text

            # TODO : Using spacy entity recognition, find the country/region + map it to ISO code

            for new_link in xml.find_all("a"):
                href = new_link.get("href")
                if href.endswith(".pdf"):
                    if href.startswith("/"):
                        href = "http://www.ipcinfo.org" + href
                    # For each download, based on the pdf's name (report/annexes/snapshots, etc...), indicate the type of file in a metadata file
                    document_type = "unknown_type"
                    pdf_filename = href.split("/")[-1]
                    for potential_document_type in ["snapshot", "report", "annexes"]:
                        if potential_document_type in pdf_filename.lower():
                            document_type = potential_document_type
                    output_path = output_folder_path / pdf_filename
                    link.update(
                        {
                            "alert_page_title": alert_page_title,
                            "pdf_url": href,
                            "document_type": document_type,
                            "pdf_local_path": str(output_path),
                        }
                    )
                    # Download the items
                    get_pdf_from_url(href, output_path=output_path)
                    documents.append(copy.copy(link))

    # save the resulting csv of metadata
    df = pd.DataFrame.from_dict(documents)
    df.to_csv(output_metadata_path, sep=";", index=False)
    return df

## Scrape them all

In [7]:
ipc_alerts_hyperlink = sources.ipc_alerts_url
output_folder_path = sources.ipc_alerts_folder_path
output_metadata_path = sources.ipc_alerts_metadata_path

# Find all hyperlinks for alerts on the page,
links = get_all_alerts_hyperlinks(ipc_alerts_hyperlink)
df = get_all_documents_from_alerts_references(links)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:46<00:00,  2.20s/it]


In [8]:
df.head()

,title,url,alert_page_title,pdf_url,document_type,pdf_local_path
0,Over 13 million people in Afghanistan likely t...,http://www.ipcinfo.org/ipcinfo-website/alerts-...,Afghanistan: Over 11 million people acutely fo...,http://www.ipcinfo.org/fileadmin/user_upload/i...,snapshot,C:\Users\charl\Repositories\batch8_actioncontr...
1,Over 13 million people in Afghanistan likely t...,http://www.ipcinfo.org/ipcinfo-website/alerts-...,Afghanistan: Over 11 million people acutely fo...,http://www.ipcinfo.org/fileadmin/user_upload/i...,report,C:\Users\charl\Repositories\batch8_actioncontr...
2,Over 13 million people in Afghanistan likely t...,http://www.ipcinfo.org/ipcinfo-website/alerts-...,Afghanistan: Over 11 million people acutely fo...,http://www.ipcinfo.org/fileadmin/user_upload/i...,report,C:\Users\charl\Repositories\batch8_actioncontr...
3,Over 13 million people in Afghanistan likely t...,http://www.ipcinfo.org/ipcinfo-website/alerts-...,Afghanistan: Over 11 million people acutely fo...,http://www.ipcinfo.org/fileadmin/user_upload/i...,snapshot,C:\Users\charl\Repositories\batch8_actioncontr...
4,Over 13 million people in Afghanistan likely t...,http://www.ipcinfo.org/ipcinfo-website/alerts-...,Afghanistan: Over 11 million people acutely fo...,http://www.ipcinfo.org/fileadmin/user_upload/i...,unknown_type,C:\Users\charl\Repositories\batch8_actioncontr...
